In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.feature_extraction.text import CountVectorizer
from  sklearn import linear_model, tree
import sklearn.ensemble as se
from sklearn import svm
from category_encoders import *
from bisect import bisect_left


from scipy.sparse import hstack
import numpy as np
import pickle
import sys

## Reading and pre-processing data

In [ ]:
'''
The app predicts the product price given a description and meta information about the product
Let's use the given information in the collected data and how can we use it:

- Restaurant information: this will be discarded, if the user knows the restaurant it will be a simple lookup,
no need for predictio
- Product information: this is the most important information we can extract, 
we will try using these fields for prediction: menu_category, product_name, product_description, 
dietary_characteristics, cuisine_characteristics, taste_characteristics, preparation_style_characteristics, 
dish_type_characteristics and ingredients
- Place you want to eat/order: this also can be very useful. 
From a personal experience: eating in Berlin is more expensive than eating in Aachen :)
Fields: postcode,latitude,longitude

Let's classify the features now using their types --> different variable types
- Fields with free text: menu_category, product_name, product_description
- Fields with a list of text values: dietary_characteristics, cuisine_characteristics, taste_characteristics, 
cuisine_characteristics, taste_characteristics, preparation_style_characteristics, dish_type_characteristics
ingredients
- Fields with categories: postcode, city_id
- Fields with numbers: latitude, longitude
'''

def conv_list(x):
    '''
        Helper to convert string list fields to actual lists
        Args:
            x: A string representing the list
        Returns: list(str): A sorted list of strings converted to lower case
    '''
    try:
        #transforming string to string list 
        return eval(x)
        #sorting the list (TODO: remove if sure that it is already sorted)
        #return sorted([s.lower() for s in l])
    except SyntaxError as e:
        return np.nan

str_fields = ['product_name', 'menu_category', 'product_description']
list_fields = ['dietary_characteristics', 'cuisine_characteristics', 'taste_characteristics', \
               'preparation_style_characteristics', 'dish_type_characteristics', 'ingredients']
cat_fields = ['postcode', 'city_id']
num_fields = ['latitude', 'longitude']
pred_fields = ['price']

feature_fields = str_fields+list_fields+cat_fields+num_fields

converters = dict(zip(list_fields, [conv_list] * len(list_fields)))

data = pd.read_csv('dataset_sub.csv', sep=';', usecols=feature_fields + pred_fields, \
                  converters=converters)

'''
Analysing missing data
- Remove columns with moree than 65% missing data
'''

for feature_type in str_fields, list_fields, cat_fields, num_fields:
    for col in feature_type:
        percent_missing = float(data[col].isnull().sum()) / float(data.shape[0])
        #print col, 'is missing', percent_missing, '% of the data'
        if percent_missing > 0.65:
            print 'Removing column', col
            data.drop(col, axis=1)
            feature_type.remove(col)
            feature_fields.remove(col)
print

#removing all rows with nan in log or lattitude
data.dropna(subset=num_fields, inplace=True)

'''
Splitting the dataset to train, dev and test sets
We will try many data transformations --> better to keep test data for final tets to avoid "overfitting" on dev data
'''

x_train, x_all_test, y_train, y_all_test = train_test_split(data[feature_fields], data['price'], \
                                                            test_size=0.2, random_state=1234)
x_dev, x_test, y_dev, y_test = train_test_split(x_all_test, y_all_test,test_size=0.5, random_state=1234)

print 'Summary of used data:'
print 'Train data size', x_train.shape[0], 'rows'
print 'Dev data size', x_dev.shape[0], 'rows'
print 'Test data size', x_test.shape[0], 'rows'
print

'''
Using RandomForestRegressor with 3 trees for testing the data tranformations
'''

clf = se.RandomForestRegressor(n_estimators=3, n_jobs=-1)


'''
Set these to True to run the testing of the different features 
If set to True it will take a lot of time for hyperparameter search

'''

test_str = False
test_list = False
test_cat = False
test_num = False
test_regressors = False

In [ ]:
'''
Let's start by transforming the string fields, these are free text fields so we will use a text analyzer
Bag of n-grams can be used in this case: 
The context is important in this case, e.g. Pizza Salami vs Pizza al Tonno
'''

# {'column_name', 'param_value'}
best_solutions = {}
# {'column_name', 'best_mse'}
best_results = dict(zip(feature_fields, [sys.float_info.max] * len(feature_fields)))
best_results['lat_long'] = sys.float_info.max
best_results['regressor'] = sys.float_info.max

if test_str:
    for n in range(1, 3):
        #strip_accents: using unicode gave better results with German
        vectorizer = CountVectorizer(ngram_range=(1, n), min_df=1, strip_accents='unicode')
        for col in str_fields:
            x_train_feats = vectorizer.fit_transform(x_train[col].replace(np.nan, '', regex=True))
            x_dev_feats = vectorizer.transform(x_dev[col].replace(np.nan, '', regex=True))

            clf.fit(x_train_feats, y_train)
            train_hyp = clf.predict(x_train_feats)
            dev_hyp = clf.predict(x_dev_feats)
            mse_dev = mean_squared_error(y_dev, dev_hyp)
            if mse_dev < best_results[col]:
                best_results[col] = mse_dev
                best_solutions[col] = n
            print 'Results using feature', col, 'and n-gram level', n
            print 'Train MSE=', mean_squared_error(y_train, train_hyp)
            print 'Dev MSE=', mse_dev
            print 'Dev R2 score=',r2_score(y_dev, dev_hyp)
            print

    for col in str_fields:
        print 'best n-gram for column', col, 'is', best_solutions[col]
else:
    best_solutions['product_name'] = 1
    best_solutions['menu_category'] = 1
    best_solutions['product_description'] = 2
    
'''
1-grams were slightly better since the context is not large, will continue with that
- product_name and product_description are more important here but they may contain redundant information
- menu_category: seems to contain some useful information
Let's test the combination and visualize the feature importance
'''

x_train_feats = None
x_dev_feats = None
x_test_feats = None

for col in str_fields:
    n_gram_level = best_solutions[col]
    vectorizer = CountVectorizer(ngram_range=(1, n_gram_level), min_df=1, strip_accents='unicode')
    x_train_col = vectorizer.fit_transform(x_train[col].replace(np.nan, '', regex=True))
    x_dev_col = vectorizer.transform(x_dev[col].replace(np.nan, '', regex=True))
    x_test_col = vectorizer.transform(x_test[col].replace(np.nan, '', regex=True))
    if x_train_feats is None:
        x_train_feats = x_train_col
        x_dev_feats = x_dev_col
        x_test_feats = x_test_col
    else:
        x_train_feats = hstack([x_train_feats, x_train_col])
        x_dev_feats = hstack([x_dev_feats, x_dev_col])
        x_test_feats = hstack([x_test_feats, x_test_col])
        
clf.fit(x_train_feats, y_train)
train_hyp = clf.predict(x_train_feats)
dev_hyp = clf.predict(x_dev_feats)

base_dev_mse = mean_squared_error(y_dev, dev_hyp)
print 'Results using bag of words features'
print 'Train MSE=', mean_squared_error(y_train, train_hyp)
print 'Dev MSE=', base_dev_mse
print 'Dev R2 score=',r2_score(y_dev, dev_hyp)


In [ ]:
'''
Let's go now further with fields containing string lists
We will try them one by one using categorical features
'''

def contains(a, v):
    '''
        Helper to search value x in array a using binary search
        Args:
            a: sorted array
            v: value to search
        Returns: Boolean: True if v exists in a
    '''
    i = bisect_left(a, v)
    if i != len(a) and a[i] == v:
        return i
    return -1

def indicatorFeature(in_list):
    '''
        Helper for one hot encoding for the lists
        Args:
            x: input list of strings
        Returns: list(string): one hot encoding of list
    '''
    result = [0] * len(cols)
    if not isinstance(in_list, list): return result
    for value in in_list:
        i = contains(cols, value)
        if i >= 0:
            result[i] = 1
    return result

transformers = [BackwardDifferenceEncoder, BinaryEncoder, \
                HashingEncoder, HelmertEncoder, OneHotEncoder, \
                OrdinalEncoder, PolynomialEncoder, SumEncoder, \
                BaseNEncoder, LeaveOneOutEncoder]

if test_list:
    for col in list_fields:
        values = []
        for v in x_train[col].values.tolist():
            if v is np.nan: continue
            values += [s for s in v]
        cols = sorted(set(values))

        x_train_feats_col = x_train[col].apply(indicatorFeature).apply(pd.Series)
        x_dev_feats_col = x_dev[col].apply(indicatorFeature).apply(pd.Series)

        for transformer in transformers:
            tr = transformer()
            tr.fit(x_train_feats_col, y_train)
            x_train_feats_col = tr.transform(x_train_feats_col)
            x_dev_feats_col = tr.transform(x_dev_feats_col)

            if x_train_feats_col.shape[1] is not x_dev_feats_col.shape[1]:
                print 'Ignoring', str(transformer), 'for column', col
                continue

            x_train_feats_test = hstack([x_train_feats, x_train_feats_col])
            x_dev_feats_test = hstack([x_dev_feats, x_dev_feats_col])

            clf.fit(x_train_feats_test, y_train)
            train_hyp = clf.predict(x_train_feats_test)
            dev_hyp = clf.predict(x_dev_feats_test)
            
            mse_dev = mean_squared_error(y_dev, dev_hyp)
            if mse_dev < best_results[col]:
                best_results[col] = mse_dev
                best_solutions[col] = tr
                
            print 'Results using indicator features for column', col, 'with transformer', str(transformer)
            print 'Train MSE=', mean_squared_error(y_train, train_hyp)
            print 'Dev MSE=', mean_squared_error(y_dev, dev_hyp)
            print 'Dev R2 score=',r2_score(y_dev, dev_hyp)
            print
            
    for col in list_fields:
        if col in best_solutions:
            print 'best transformer for column', col, 'is', best_solutions[col]
            if best_results[col] > base_dev_mse:
                print 'Warning: worse results than baseline with column', col 
        else:
            print 'Warning: transforming', col, 'failed with all transformers'
            list_fields.remove(col)
                
else:
    best_solutions['cuisine_characteristics'] = OrdinalEncoder()
    best_solutions['taste_characteristics'] = BackwardDifferenceEncoder() 
    best_solutions['preparation_style_characteristics'] = LeaveOneOutEncoder()
    best_solutions['dish_type_characteristics'] = PolynomialEncoder()
    list_fields.remove('ingredients')
    
'''
All features did not hurt performance or gave better results 
we will keep them and refince with feature selection later
'''

In [ ]:

'''
We will work now with categorical features with different categorical feature representations
Using the category_encoders library module as well
'''

if test_cat:
    for col in cat_fields:
        for transformer in transformers:
            tr = transformer()
            tr.fit(x_train[col].to_frame(), y_train)
            x_train_feats_col = tr.transform(x_train[col].to_frame())
            x_dev_feats_col = tr.transform(x_dev[col].to_frame())

            if x_train_feats_col.shape[1] is not x_dev_feats_col.shape[1]:
                print 'Ignoring', str(transformer), 'for column', col
                continue

            x_train_feats_col.fillna(0, inplace=True)
            x_dev_feats_col.fillna(0, inplace=True)

            x_train_feats_test = hstack([x_train_feats, x_train_feats_col])
            x_dev_feats_test = hstack([x_dev_feats, x_dev_feats_col])

            clf.fit(x_train_feats_test, y_train)
            train_hyp = clf.predict(x_train_feats_test)
            dev_hyp = clf.predict(x_dev_feats_test)

            mse_dev = mean_squared_error(y_dev, dev_hyp)
            if mse_dev < best_results[col]:
                best_results[col] = mse_dev
                best_solutions[col] = tr

            print 'Results using feature', col, 'with transformer', str(transformer)
            print 'Train MSE=', mean_squared_error(y_train, train_hyp)
            print 'Dev MSE=', mse_dev
            print 'Dev R2 score=',r2_score(y_dev, dev_hyp)
            print
            
    for col in cat_fields:
        print 'best transformer for column', col, 'is', best_solutions[col]
else:
    best_solutions['postcode'] = BinaryEncoder()
    best_solutions['city_id'] = BinaryEncoder()

'''
Best results for the categorical features:
postcode: BackwardDifferenceEncoder
city_id: LeaveOneOutEncoder
'''

In [ ]:
'''
We will continue now with the logitude/lattitude features
The idea is to have clusters places using the 
'''
from sklearn.cluster import DBSCAN
import scipy as sp
from haversine import haversine

x_train_places = np.radians(x_train[num_fields])
x_dev_places = np.radians(x_dev[num_fields])
x_test_places = np.radians(x_test[num_fields])

kms_per_radian = 6371.0088
epsilon = 1.0 / kms_per_radian
dbscan_model = DBSCAN(eps=epsilon, min_samples=1, algorithm='ball_tree', metric='haversine')

x_train_clusters = dbscan_model.fit_predict(x_train_places)
x_train_clusters = np.resize(x_train_clusters, (x_train_clusters.shape[0], 1))

def dbscan_predict(points, metric=haversine):
    '''
        Helper to find closest cluster
        Args:
            dbscan_model: trained sklearn DBSCAN model
            X_new np.array: lat/long in radians
            metric: used metric for comparing points
        Returns: np.array: cluster values
    '''
    # Result is noise by default
    cluster = np.ones(shape=len(points), dtype=int)*-1

    # Iterate all input samples for a label
    for j, point in enumerate(points):
        # Find a core sample closer than EPS
        for i, x_core in enumerate(dbscan_model.components_):
            if metric(point, x_core) < dbscan_model.eps:
                # Assign label
                cluster[j] = dbscan_model.labels_[dbscan_model.core_sample_indices_[i]]
                break
    return cluster

x_dev_clusters = dbscan_predict(x_dev_places.as_matrix())
x_dev_clusters = np.resize(x_dev_clusters, (x_dev_clusters.shape[0], 1))

x_test_clusters = dbscan_predict(x_test_places.as_matrix())
x_test_clusters = np.resize(x_test_clusters, (x_test_clusters.shape[0], 1))

if test_num:
    for transformer in transformers:
            tr = transformer()
            tr.fit(x_train_clusters, y_train)
            x_train_feats_col = tr.transform(x_train_clusters)
            x_dev_feats_col = tr.transform(x_dev_clusters)
             
            if x_train_feats_col.shape[1] is not x_dev_feats_col.shape[1]:
                print 'Ignoring', str(transformer), 'for latitude/longitude'
                continue
                
            x_train_feats_test = hstack([x_train_feats, x_train_feats_col])
            x_dev_feats_test = hstack([x_dev_feats, x_dev_feats_col])

            clf.fit(x_train_feats_test, y_train)
            train_hyp = clf.predict(x_train_feats_test)
            dev_hyp = clf.predict(x_dev_feats_test)
            
            mse_dev = mean_squared_error(y_dev, dev_hyp)
            if mse_dev < best_results['lat_long']:
                best_results['lat_long'] = mse_dev
                best_solutions['lat_long'] = tr

            print 'Results using logitude/lattitude with transformer', transformer
            print 'Train MSE=', mean_squared_error(y_train, train_hyp)
            print 'Dev MSE=', mse_dev
            print 'Dev R2 score=',r2_score(y_dev, dev_hyp)
            print
    print 'best transformer for column', col, 'is', str(best_solutions['lat_long'])
else:
    best_solutions['lat_long'] = HashingEncoder()


In [ ]:
'''
Testing different regressors using the selected columns / parameters

'''
for col in list_fields:
    values = []
    for v in x_train[col].values.tolist():
        if v is np.nan: continue
        values += [s for s in v]
    cols = sorted(set(values))

    x_train_feats_col = x_train[col].apply(indicatorFeature).apply(pd.Series)
    x_dev_feats_col = x_dev[col].apply(indicatorFeature).apply(pd.Series)
    x_test_feats_col = x_test[col].apply(indicatorFeature).apply(pd.Series)
    transformer = best_solutions[col]
    transformer.fit(x_train_feats_col, y_train)

    x_train_feats_col = transformer.transform(x_train_feats_col)
    x_dev_feats_col = transformer.transform(x_dev_feats_col)
    x_test_feats_col = transformer.transform(x_test_feats_col)

    if x_train_feats_col.shape[1] is not x_dev_feats_col.shape[1] or \
            x_dev_feats_col.shape[1] is not x_test_feats_col.shape[1]:
        print 'Warning: Ignoring column', col
        continue

    x_train_feats = hstack([x_train_feats, x_train_feats_col])
    x_dev_feats = hstack([x_dev_feats, x_dev_feats_col])
    x_test_feats = hstack([x_test_feats, x_test_feats_col])

for col in cat_fields:
    transformer = best_solutions[col]
    transformer.fit(x_train[col].to_frame(), y_train)
    x_train_feats_col = transformer.transform(x_train[col].to_frame())
    x_dev_feats_col = transformer.transform(x_dev[col].to_frame())
    x_test_feats_col = transformer.transform(x_test[col].to_frame())

    if x_train_feats_col.shape[1] is not x_dev_feats_col.shape[1]:
        print 'Warning: Ignoring column', col
        continue

    x_train_feats_col.fillna(0, inplace=True)
    x_dev_feats_col.fillna(0, inplace=True)
    x_test_feats_col.fillna(0, inplace=True)

    x_train_feats = hstack([x_train_feats, x_train_feats_col])
    x_dev_feats = hstack([x_dev_feats, x_dev_feats_col])
    x_test_feats = hstack([x_test_feats, x_test_feats_col])

transformer = best_solutions['lat_long']
transformer.fit(x_train_clusters, y_train)
x_train_feats_col = transformer.transform(x_train_clusters)
x_dev_feats_col = transformer.transform(x_dev_clusters)
x_dev_test_col = transformer.transform(x_test_clusters)

if x_train_feats_col.shape[1] is not x_dev_feats_col.shape[1]:
    print 'Ignoring', str(transformer), 'for latitude/longitude'
else:
    x_train_feats = hstack([x_train_feats, x_train_feats_col])
    x_dev_feats = hstack([x_dev_feats, x_dev_feats_col])
    x_test_feats = hstack([x_test_feats, x_dev_test_col])

'''
Trying different regressors, RandomForestRegressor performs better on this data
svm.SVR: Support Vector Classification extended to solve regression problems
linear_model.SGDRegressor: Linear model fitted by minimizing a regularized empirical loss with SGD
se.RandomForestRegressor: A random forest is a meta estimator that fits a number of classifying decision 
trees on various sub-samples
'''

if test_regressor:
    regressors = [
        svm.SVR(),
        linear_model.SGDRegressor(),
        se.RandomForestRegressor(n_jobs=-1)]

    for regressor in regressors:
        regressor.fit(x_train_feats, y_train)
        train_hyp = regressor.predict(x_train_feats)
        dev_hyp = regressor.predict(x_dev_feats)

        mse_dev = mean_squared_error(y_dev, dev_hyp)
        if mse_dev < best_results['regressor']:
            best_results['regressor'] = mse_dev
            best_solutions['regressor'] = regressor

        print 'Results with regressor', type(regressor).__name__
        print 'Train MSE=', mean_squared_error(y_train, train_hyp)
        print 'Dev MSE=', mse_dev
        print 'Dev R2 score=',r2_score(y_dev, dev_hyp)
        print
    print 'best regressor is', type(best_solutions['regressor']).__name__
else:
    best_solutions['regressor'] = se.RandomForestRegressor(n_jobs=-1)

def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

save_obj(best_solutions, 'models/best_config')

In [ ]:
print x_train_feats.shape, x_test_feats.shape, x_dev_feats.shape

In [ ]:
#Testing and plotting
regressor = best_solutions['regressor']
regressor.fit(x_train_feats, y_train)
test_hyp = regressor.predict(x_test_feats)
print 'Results with regressor', type(regressor).__name__
print 'Test MSE=', mean_squared_error(y_test, test_hyp)
print 'Test R2 score=',r2_score(y_dev, dev_hyp)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, _ = plt.subplots(1, 1, sharey=True)
p = pd.DataFrame({feature: x_test[feature], 'Hyp':test_hyp })
p1 = pd.DataFrame({feature: x_test[feature], 'Ref':y_test })

#p = p.set_index(feature, drop=False).groupby(feature)
plt.ylabel('Price')
plt.plot(p.index.values, p['Hyp'], '.r', label="Hypothesized prices")
plt.plot(p1.index.values, p1['Ref'], '.b', label="Reference prices")
fig.set_figheight(5)
fig.set_figwidth(15)

plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3, ncol=2, mode="expand", borderaxespad=0.)

plt.show()
